In [2]:
import Augmentor
import numpy as np
import os
import glob
import random
import collections
from PIL import Image
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
import matplotlib.pyplot as plt

In [3]:
root_directory = "C:\\Users\\ASUS\\project\\Plant-Leaf-Recognition\\Data_Augumentation"

folders = []
for f in glob.glob(os.path.join(root_directory, '*')):
    if os.path.isdir(f):
        folders.append(os.path.abspath(f))

print("Folders (classes) found: %s " % [os.path.split(x)[1] for x in folders])


Folders (classes) found: ['leaf1', 'leaf10', 'leaf11', 'leaf12', 'leaf13', 'leaf14', 'leaf15', 'leaf2', 'leaf3', 'leaf4', 'leaf5', 'leaf6', 'leaf7', 'leaf8', 'leaf9'] 


In [4]:
pipelines = {}
for folder in folders:
    print("Folder %s:" % (folder))
    pipelines[os.path.split(folder)[1]] = (Augmentor.Pipeline(folder))
    print("\n----------------------------\n")

for p in pipelines.values():
    print("Class %s has %s samples." % (p.augmentor_images[0].class_label, len(p.augmentor_images)))

pipelines


Folder C:\Users\ASUS\project\Plant-Leaf-Recognition\Data_Augumentation\leaf1:
Initialised with 75 image(s) found.
Output directory set to C:\Users\ASUS\project\Plant-Leaf-Recognition\Data_Augumentation\leaf1\output.
----------------------------

Folder C:\Users\ASUS\project\Plant-Leaf-Recognition\Data_Augumentation\leaf10:
Initialised with 75 image(s) found.
Output directory set to C:\Users\ASUS\project\Plant-Leaf-Recognition\Data_Augumentation\leaf10\output.
----------------------------

Folder C:\Users\ASUS\project\Plant-Leaf-Recognition\Data_Augumentation\leaf11:
Initialised with 75 image(s) found.
Output directory set to C:\Users\ASUS\project\Plant-Leaf-Recognition\Data_Augumentation\leaf11\output.
----------------------------

Folder C:\Users\ASUS\project\Plant-Leaf-Recognition\Data_Augumentation\leaf12:
Initialised with 75 image(s) found.
Output directory set to C:\Users\ASUS\project\Plant-Leaf-Recognition\Data_Augumentation\leaf12\output.
----------------------------

Folder C:\

{'leaf1': <Augmentor.Pipeline.Pipeline at 0x2503b436d10>,
 'leaf10': <Augmentor.Pipeline.Pipeline at 0x2503b380650>,
 'leaf11': <Augmentor.Pipeline.Pipeline at 0x2503b4ab290>,
 'leaf12': <Augmentor.Pipeline.Pipeline at 0x2503b5aba10>,
 'leaf13': <Augmentor.Pipeline.Pipeline at 0x2503b513650>,
 'leaf14': <Augmentor.Pipeline.Pipeline at 0x2503b513690>,
 'leaf15': <Augmentor.Pipeline.Pipeline at 0x2503b522090>,
 'leaf2': <Augmentor.Pipeline.Pipeline at 0x2503b4ff610>,
 'leaf3': <Augmentor.Pipeline.Pipeline at 0x2503b4bbed0>,
 'leaf4': <Augmentor.Pipeline.Pipeline at 0x2503b529cd0>,
 'leaf5': <Augmentor.Pipeline.Pipeline at 0x2503b5d2450>,
 'leaf6': <Augmentor.Pipeline.Pipeline at 0x2503b64c950>,
 'leaf7': <Augmentor.Pipeline.Pipeline at 0x2503b4b8750>,
 'leaf8': <Augmentor.Pipeline.Pipeline at 0x2503b5d2650>,
 'leaf9': <Augmentor.Pipeline.Pipeline at 0x2503b5813d0>}

In [5]:
for pipeline in pipelines.values():
    pipeline.rotate(probability=0.75, max_left_rotation=10, max_right_rotation=10)
    pipeline.flip_left_right(probability=0.8)
    pipeline.skew(probability=0.4)
    pipeline.random_distortion(probability=0.5, grid_width=3, grid_height=7,magnitude=2)
    pipeline.crop_centre(probability=0.1,percentage_area=0.8)
    pipeline.sample(333)

integer_labels = {
    'leaf1': 0,
    'leaf2': 1,
    'leaf3': 2,
    'leaf4': 3,
    'leaf5': 4,
    'leaf6': 5,
    'leaf7': 6,
    'leaf8': 7,
    'leaf9': 8,
    'leaf10': 9,
    'leaf11': 10,
    'leaf12': 11,
    'leaf13': 12,
    'leaf14': 13,
    'leaf15': 14
}

PipelineContainer = collections.namedtuple(
    'PipelineContainer',
    'label label_integer label_categorical pipeline generator'
)

pipeline_containers = []

for label, pipeline in pipelines.items():
    #label_categorical = np.zeros(len(pipelines), dtype=np.int)
    
    label_categorical = np.zeros(len(pipelines), dtype=int)
    
    label_categorical[integer_labels[label]] = 1
    pipeline_containers.append(
        PipelineContainer(
            label,
            integer_labels[label],
            label_categorical,
            pipeline,
            pipeline.keras_generator(batch_size=1)
        )
    )



def multi_generator(pipeline_containers, batch_size):
    X = []
    y = []
    for i in range(batch_size):
        pipeline_container = random.choice(pipeline_containers)
        image, _ = next(pipeline_container.generator)
        image = image.reshape((224,224,3)) # Or (1, 28, 28) for channels_first, see Keras' docs.
        X.append(image)
        y.append(pipeline_container.label_categorical) # Or label_integer if required by network
        X = np.asarray(X)
        y = np.asarray(y)
    yield X, y
            
multi_generator(pipeline_containers,1)


Processing <PIL.Image.Image image mode=RGB size=984x1643 at 0x2503B581310>: 100%|█| 333/333 [00:15<00:00, 20.89 Samples
Processing <PIL.Image.Image image mode=RGB size=1922x3147 at 0x25049E85C10>: 100%|█| 333/333 [00:46<00:00,  7.24 Sample
Processing <PIL.Image.Image image mode=RGB size=838x1520 at 0x2504AEB3150>: 100%|█| 333/333 [00:11<00:00, 29.77 Samples
Processing <PIL.Image.Image image mode=RGB size=987x1562 at 0x25045EF4750>: 100%|█| 333/333 [00:22<00:00, 14.69 Samples
Processing <PIL.Image.Image image mode=RGB size=1307x1918 at 0x25041FD0350>: 100%|█| 333/333 [00:16<00:00, 20.23 Sample
Processing <PIL.Image.Image image mode=RGB size=1017x1342 at 0x2503D1A8BD0>: 100%|█| 333/333 [00:20<00:00, 16.48 Sample
Processing <PIL.Image.Image image mode=RGB size=721x1082 at 0x2503B52A2D0>: 100%|█| 333/333 [00:14<00:00, 22.52 Samples
Processing <PIL.Image.Image image mode=RGB size=1558x1994 at 0x2505FC2F210>: 100%|█| 333/333 [01:05<00:00,  5.11 Sample
Processing <PIL.Image.Image image mode=R

<generator object multi_generator at 0x000002503B6297A0>